In [2]:
import pandas as pd
import numpy as np
%run -i "Cash_report_dict.ipynb"
%run -i "Cash_report_func.ipynb"
import datetime
import time
from collections import Counter
import datetime
import xlwt
import xlrd
import xdrlib,sys

In [3]:
import inspect, os
BASE = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
BASE

'/Users/olivia/Desktop/Cash Report'

In [4]:
os.listdir()

['DailyBAL_consolidated.xls',
 '.DS_Store',
 'testfinal00.ipynb',
 'Cash_report_dict.ipynb',
 'Cash_report_func.ipynb',
 '.ipynb_checkpoints',
 'cash_event_20190430interest(out).csv']

In [5]:
df_in=pd.read_excel(os.path.join(BASE, 'DailyBAL_consolidated.xls'))
df_tmpt=pd.read_csv(os.path.join(BASE, 'cash_event_20190430interest(out).csv'), skiprows = 3)
df_in

,Account Name,Account Number,Account Owner Reference,Bank Code,Beneficiary Account Number,Business Date,Channel,Cheque Number,Client ID,Closing Available Balance,...,Transaction Date,Transaction Details,Transaction Reference,Transaction Short Description,Transaction Type,VA Flag,VA Number,VA Payment Reference,Value Date,FX Rate
0,PPAL-FAM GO PLUS FUND - S&R USD,104011645,NaN,SCBLSGSGXXX,104011645.0,09/04/2019,OPAL,NaN,NaN,"1,749,536.90",...,09/04/2019,AXA INSURANCE PT AXA INSURANCE PTE. LTD. FROM ...,IT10501904094714,AXA INSURANCE PT AXA INSURANCE PTE. LTD.,NaN,NaN,NaN,NaN,09/04/2019,1.00000
1,PPAL-FAM GO PLUS FUND - USD,104011637,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,"1,443,708.34",...,09/04/2019,83847-RDO:46905927:1-PCHSTLMNTRCV-H BNC Trans...,0013791,83847-RDO:46905927:1-PCHSTLMNTRCV-H,NaN,NaN,NaN,NaN,09/04/2019,1.00000
2,PPAL-FAM GO PLUS FUND - USD,104011637,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,"1,443,708.34",...,09/04/2019,AIM8916-RDO:46926899:1-SUBSCRIPTION BNC Trans...,0014174,AIM8916-RDO:46926899:1-SUBSCRIPTION,NaN,NaN,NaN,NaN,09/04/2019,1.00000
3,PPAL-FAM GO PLUS FUND - USD,104011637,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,"1,443,708.34",...,09/04/2019,AIM8781-RDO:46847782:1-SUBSCRIPTION BNC Trans...,0014175,AIM8781-RDO:46847782:1-SUBSCRIPTION,NaN,NaN,NaN,NaN,09/04/2019,1.00000
4,PPAL-FAM GO PLUS FUND - USD,104011637,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,"1,443,708.34",...,09/04/2019,AIM8918-RDO:47002237:1-SUBSCRIPTION BNC Trans...,0014183,AIM8918-RDO:47002237:1-SUBSCRIPTION,NaN,NaN,NaN,NaN,09/04/2019,1.00000
5,PPAL-FAM GO PLUS FUND - USD,104011637,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,"1,443,708.34",...,09/04/2019,FSINV 524922 FEB-GBT:53115:1-INTRNL BNC Trans...,0014503,FSINV 524922 FEB-GBT:53115:1-INTRNL,NaN,NaN,NaN,NaN,09/04/2019,1.00000
6,PPAL-FAM GO PLUS FUND - JPY,104011629,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00897
7,PPAL-FAM GO PLUS FUND - EUR,104011610,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.12625
8,PPAL-FAM GO PLUS FUND - HKD,104011602,NaN,SCBLSGSGXXX,NaN,09/04/2019,NaN,NaN,NaN,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12747
9,PPAL-FAM GO PLUS FUND - S&R SGD,100778984,NaN,SCBLSGSGXXX,850570027.0,09/04/2019,OPAL,NaN,NaN,505.77,...,09/04/2019,SSG03401C0001546-00001-PIGSG03401A04216 STS ...,SE10501904090289,SSG03401C0001546-00001-PIGSG03401A04216 STS,NaN,NaN,NaN,NaN,09/04/2019,0.73817


In [6]:
df_in=df_in.dropna(subset=['Dr / Cr Indicator'])

In [7]:
df_out=pd.DataFrame(columns=df_tmpt.columns)

In [8]:
# #in_principal
df_out['#in_principal'] = df_in['Account Name'].apply(in_principal)

In [9]:
# in_book
currency=[i[-3:] for i in df_in['Account Name']]
def in_book(Acc_nm):
    for i in range(len(Acc_nm)):
        if 'S&R' in Acc_nm:
            return "_SNR_{}".format(currency[i])
        else:
            return "_BK"
df_out['in_book'] = df_out['#in_principal'] + df_in['Account Name'].apply(in_book)
#df_out

In [10]:
# in_strategy
df_out['in_strategy'] = 'NONE'

# in_prime_broker
df_out['in_prime_broker'] = 'SCB'

# in_pb_account
df_out['in_pb_account'] = df_out['#in_principal'] + '_' + df_out['in_prime_broker'] + '_ACCTID' 

# in_pl_book_ccy_xrate
df_out['in_pl_book_ccy_xrate'] = ''

# in_pl_book_ccy_xrate_mdiv
df_out['in_pl_book_ccy_xrate_mdiv'] = ''

# in_cpty
df_out['in_cpty'] = df_out['in_prime_broker']

# in_narrative
df_out['in_narrative'] = ''

# in_pay_rcv
df_out['in_pay_rcv'] = df_in['Dr / Cr Indicator'].apply(in_payrcv)

# in_amt
df_out['in_amt'] = df_in['Payment Amount']

# in_ccy
df_out['in_ccy'] = currency

# 
df_out[['in_ident_type',
       'in_ext_ident', 'in_inst_class', 'in_price_divisor', 'in_type',
       'in_sec_event_id', 'in_behaviour_type', 'in_realise_difference_ind',
       'in_logon', 'in_canc_amnd_ind', 'in_cnev_id']] = ''

# in_event_type
df_out = in_event_type(df_out = df_out, df_in = df_in)

In [11]:
isinstance(df_in['Business Date'][5], datetime.datetime)

False

In [12]:
df_out['in_event_date'] = df_in['Business Date'].apply(get_bzday)
df_out['in_value_date'] = df_out['in_event_date']

df_out.reset_index(inplace=True, drop = True)
df_out

,#in_principal,in_book,in_strategy,in_prime_broker,in_pb_account,in_pl_book_ccy_xrate,in_pl_book_ccy_xrate_mdiv,in_cpty,in_event_type,in_narrative,...,in_ext_ident,in_inst_class,in_price_divisor,in_type,in_sec_event_id,in_behaviour_type,in_realise_difference_ind,in_logon,in_canc_amnd_ind,in_cnev_id
0,FGOPLUS,FGOPLUS_SNR_USD,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
1,FGOPLUS,FGOPLUS_BK,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
2,FGOPLUS,FGOPLUS_BK,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
3,FGOPLUS,FGOPLUS_BK,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
4,FGOPLUS,FGOPLUS_BK,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
5,FGOPLUS,FGOPLUS_BK,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,FUND_SVCES_FEE,,...,,,,,,,,,,
6,FGOPLUS,FGOPLUS_SNR_USD,NONE,SCB,FGOPLUS_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
7,FGO,FGO_SNR_USD,NONE,SCB,FGO_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
8,FGO,FGO_SNR_USD,NONE,SCB,FGO_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,
9,FGO,FGO_SNR_USD,NONE,SCB,FGO_SCB_ACCTID,,,SCB,NaN,,...,,,,,,,,,,


In [13]:
df_out.columns

Index(['#in_principal', 'in_book', 'in_strategy', 'in_prime_broker',
       'in_pb_account', 'in_pl_book_ccy_xrate', 'in_pl_book_ccy_xrate_mdiv',
       'in_cpty', 'in_event_type', 'in_narrative', 'in_pay_rcv', 'in_amt',
       'in_ccy', 'in_event_date', 'in_value_date', 'in_ident_type',
       'in_ext_ident', 'in_inst_class', 'in_price_divisor', 'in_type',
       'in_sec_event_id', 'in_behaviour_type', 'in_realise_difference_ind',
       'in_logon', 'in_canc_amnd_ind', 'in_cnev_id'],
      dtype='object')

In [216]:
# a new excel 
file = xlwt.Workbook()  
# new sheet  
sheet0 = file.add_sheet('info',cell_overwrite_ok=True)  
# write data into sheet: sheet_name.write(r,col,value)  
sheet0.write(0,0,'#!CONNECT=HK067_GNG/HK067_GNG@prod_ho3orc12_fm.world##')
sheet0.write(0,1,'Connected')  
sheet0.write(1,0,'#!MAX_ERROR=1000') 
sheet0.write(2,0,'#!OPF=TDP_LOADER_2.insert_cash_event') 

for i, col in enumerate(df_out.columns):
    cash_event_20190430interest.write(3, i, col)

for i in df_out.index:
    print('-------------------------------- {}'.format(i))
    for j in range(len(df_out.columns)):
        temp_value = df_out.iloc[i, j]
        print('# {} - {} - {}'.format(j, temp_value, type(temp_value)))        
        try:
            if isinstance(temp_value, str):
                cash_event_20190430interest.write(int(i)+4, j, temp_value)
            elif isinstance(temp_value, float):
                cash_event_20190430interest.write(int(i)+4, j, '')
            elif isinstance(temp_value, type(None)):
                cash_event_20190430interest.write(int(i)+4, j, '')
        except Exception as e:
            print(e)

# save file 
file.save('cash_event_20190430interest.csv')

-------------------------------- 0
# 0 - FGOPLUS - <class 'str'>
# 1 - FGOPLUS_SNR_USD - <class 'str'>
# 2 - NONE - <class 'str'>
# 3 - SCB - <class 'str'>
# 4 - FGOPLUS_SCB_ACCTID - <class 'str'>
# 5 -  - <class 'str'>
# 6 -  - <class 'str'>
# 7 - SCB - <class 'str'>
# 8 - nan - <class 'float'>
# 9 -  - <class 'str'>
# 10 - None - <class 'NoneType'>
# 11 - 301,832.42 - <class 'str'>
# 12 - USD - <class 'str'>
# 13 - 09/04/2019 - <class 'str'>
# 14 - 09/04/2019 - <class 'str'>
# 15 -  - <class 'str'>
# 16 -  - <class 'str'>
# 17 -  - <class 'str'>
# 18 -  - <class 'str'>
# 19 -  - <class 'str'>
# 20 -  - <class 'str'>
# 21 -  - <class 'str'>
# 22 -  - <class 'str'>
# 23 -  - <class 'str'>
# 24 -  - <class 'str'>
# 25 -  - <class 'str'>
-------------------------------- 1
# 0 - FGOPLUS - <class 'str'>
# 1 - FGOPLUS_BK - <class 'str'>
# 2 - NONE - <class 'str'>
# 3 - SCB - <class 'str'>
# 4 - FGOPLUS_SCB_ACCTID - <class 'str'>
# 5 -  - <class 'str'>
# 6 -  - <class 'str'>
# 7 - SCB - <cla